# Telstra Network Disruptions- Notebook 2
    * In this notebook, we're gonna try out new feature representations and we're gonna try training different models on different parts of the dataset

In [2]:
%matplotlib inline
import IPython.core.display         
# setup output image format (Chrome works best)
IPython.core.display.set_matplotlib_formats("svg")
import matplotlib.pyplot as plt
import matplotlib
from sklearn import *
from scipy import stats
import IPython.utils.warn as warn
random.seed(100)
import json
import csv
import pickle
import numpy as np
from sklearn.ensemble import GradientBoostingClassifier
import numpy as np
import scipy.sparse
import pickle
import xgboost as xgb
import pandas as pd

/home/shivam6294/anaconda2/lib/python2.7/site-packages/sklearn/lda.py:4: DeprecationWarning: lda.LDA has been moved to discriminant_analysis.LinearDiscriminantAnalysis in 0.17 and will be removed in 0.19
  "in 0.17 and will be removed in 0.19", DeprecationWarning)
/home/shivam6294/anaconda2/lib/python2.7/site-packages/sklearn/qda.py:4: DeprecationWarning: qda.QDA has been moved to discriminant_analysis.QuadraticDiscriminantAnalysis in 0.17 and will be removed in 0.19.
  "in 0.17 and will be removed in 0.19.", DeprecationWarning)


In [3]:
#Import Pandas Dataframe
import pandas as pd

In [4]:
train = pd.read_csv('Telstra_data/train.csv')

## IMPORT ALL THE DATA

In [5]:
print train.dtypes
train.set_index([train.id],inplace=True)
print '__________________________'
print train[:10]
print 'Number of tuples:', len(train)

def get_num(z):
    return str(z).split()[1]

train['location'] = train['location'].apply(lambda x: get_num(x)) # returns DataFrame
#train = pd.get_dummies(train,columns = ['location'])

id                 int64
location          object
fault_severity     int64
dtype: object
__________________________
          id      location  fault_severity
id                                        
14121  14121  location 118               1
9320    9320   location 91               0
14394  14394  location 152               1
8218    8218  location 931               1
14804  14804  location 120               0
1080    1080  location 664               0
9731    9731  location 640               0
15505  15505  location 122               0
3443    3443  location 263               1
13300  13300  location 613               1
Number of tuples: 7381


In [6]:
train[10]

          id location  fault_severity
id                                   
14121  14121      118               1
9320    9320       91               0
14394  14394      152               1
8218    8218      931               1
14804  14804      120               0
1080    1080      664               0
9731    9731      640               0
15505  15505      122               0
3443    3443      263               1
13300  13300      613               1
8976    8976      760               1
7965    7965      519               0
10342  10342      746               0
6741    6741     1066               1
7187    7187      812               0
16780  16780      343               0
14195  14195      133               0
14025  14025      976               0
8548    8548      948               1
9566    9566      808               0
15652  15652      894               0
3931    3931      875               0
4557    4557     1024               0
3610    3610      166               0
16574  16574

In [7]:
test = pd.read_csv('Telstra_data/test.csv')

In [8]:
test['location'] = test['location'].apply(lambda x: get_num(x)) # returns DataFrame
test.set_index([test.id],inplace=True)
print test.dtypes
print '__________________________'
print test[:10]
print 'Number of tuples:', len(test)
#test = pd.get_dummies(test,columns = ['location'])

id           int64
location    object
dtype: object
__________________________
          id location
id                   
11066  11066      481
18000  18000      962
16964  16964      491
4795    4795      532
3392    3392      600
3795    3795      794
2881    2881      375
1903    1903      638
5245    5245      690
6726    6726      893
Number of tuples: 11171


In [9]:
event_type = pd.read_csv('Telstra_data/event_type.csv')
event_type.set_index([event_type.id],inplace=True)
event_type['event_type'] = event_type['event_type'].apply(lambda x: get_num(x)) # returns DataFrame

print event_type.dtypes
print '__________________________'

print 'Number of tuples:', len(event_type)

event_typex = pd.get_dummies(event_type,columns = ['event_type'])
event_typex = pd.DataFrame(event_typex.groupby(by='id').sum())

event_type = pd.DataFrame(event_type.groupby(by='id')['event_type'].count())
event_type = pd.merge(event_type, event_typex, left_index=True,right_index=True, how='left')
event_type[:20]

id             int64
event_type    object
dtype: object
__________________________
Number of tuples: 31170


,event_type,event_type_1,event_type_10,event_type_11,event_type_12,event_type_13,event_type_14,event_type_15,event_type_17,event_type_18,...,event_type_5,event_type_50,event_type_51,event_type_52,event_type_53,event_type_54,event_type_6,event_type_7,event_type_8,event_type_9
id,,,,,,,,,,,,,,,,,,,,,
1,2,0,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,2,0,0,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [10]:
resource_type = pd.read_csv('Telstra_data/resource_type.csv')
resource_type.set_index([resource_type.id],inplace=True)
resource_type['resource_type'] = resource_type['resource_type'].apply(lambda x: get_num(x)) # returns DataFrame

print resource_type.dtypes
print '__________________________'
print 'Number of tuples:',len(resource_type)


resource_typex = pd.get_dummies(resource_type,columns = ['resource_type'])
resource_typex = pd.DataFrame(resource_typex.groupby(by='id').sum())

resource_type = pd.DataFrame(resource_type.groupby(by='id')['resource_type'].count())
resource_type = pd.merge(resource_type, resource_typex, left_index=True,right_index=True, how='left')
resource_type[:20]

id                int64
resource_type    object
dtype: object
__________________________
Number of tuples: 21076


,resource_type,resource_type_1,resource_type_10,resource_type_2,resource_type_3,resource_type_4,resource_type_5,resource_type_6,resource_type_7,resource_type_8,resource_type_9
id,,,,,,,,,,,
1,2,0,0,0,0,0,0,1,0,1,0
2,1,0,0,1,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,1,0
4,1,0,0,1,0,0,0,0,0,0,0
5,1,0,0,1,0,0,0,0,0,0,0
6,1,0,0,1,0,0,0,0,0,0,0
7,1,0,0,1,0,0,0,0,0,0,0
8,1,0,0,1,0,0,0,0,0,0,0
9,1,0,0,0,0,0,0,0,0,1,0


In [11]:
severity_type = pd.read_csv('Telstra_data/severity_type.csv')
severity_type.set_index([severity_type.id],inplace=True)
severity_type['severity_type'] = severity_type['severity_type'].apply(lambda x: get_num(x)) # returns DataFrame

# print severity_type.dtypes
# print '__________________________'
# print severity_type[:10]
# print 'Number of tuples:-',len(severity_type)

# severity_type = severity_type.groupby(by='severity_type').count()
severity_type

,id,severity_type
id,,
6597,6597,2
8011,8011,2
2597,2597,2
5022,5022,1
6852,6852,1
5611,5611,2
14838,14838,1
2588,2588,1
4848,4848,1


In [12]:
log_feature = pd.read_csv('Telstra_data/log_feature.csv')
log_feature.set_index([log_feature.id],inplace=True)

log_feature['log_feature'] = log_feature['log_feature'].apply(lambda x: get_num(x)) # returns DataFrame

print log_feature.dtypes
print '__________________________'
print 'Number of tuples:-',len(log_feature)

log_featurex = pd.get_dummies(log_feature,columns = ['log_feature'])

def apply_vol(x):
    volume = x['volume']
    a = x[2:].apply(lambda b:volume if b!=0 else b+1)
    return pd.Series(a)

log_featurex = log_featurex.apply(apply_vol,axis=1)

id              int64
log_feature    object
volume          int64
dtype: object
__________________________
Number of tuples:- 58671


In [13]:
#LOG NORMALIZATION
log_featurex = log_featurex.apply(lambda x: np.log(x))
log_featurex['id'] = log_feature.index
log_featurex['log_frequency'] = 1

In [14]:
log_featurex = pd.DataFrame(log_featurex.groupby(by='id').aggregate(np.sum))
log_featurex[:20]


,log_feature_1,log_feature_10,log_feature_100,log_feature_101,log_feature_102,log_feature_103,log_feature_104,log_feature_105,log_feature_106,log_feature_107,...,log_feature_91,log_feature_92,log_feature_93,log_feature_94,log_feature_95,log_feature_96,log_feature_97,log_feature_98,log_feature_99,log_frequency
id,,,,,,,,,,,,,,,,,,,,,
1,0,0,0,0,0,0,0,0.000000,0,0,...,0,0,0,0,0,0,0,0,0,3
2,0,0,0,0,0,0,0,0.000000,0,0,...,0,0,0,0,0,0,0,0,0,5
3,0,0,0,0,0,0,0,0.000000,0,0,...,0,0,0,0,0,0,0,0,0,1
4,0,0,0,0,0,0,0,0.000000,0,0,...,0,0,0,0,0,0,0,0,0,1
5,0,0,0,0,0,0,0,0.000000,0,0,...,0,0,0,0,0,0,0,0,0,2
6,0,0,0,0,0,0,0,0.000000,0,0,...,0,0,0,0,0,0,0,0,0,2
7,0,0,0,0,0,0,0,0.000000,0,0,...,0,0,0,0,0,0,0,0,0,4
8,0,0,0,0,0,0,0,0.000000,0,0,...,0,0,0,0,0,0,0,0,0,6
9,0,0,0,0,0,0,0,3.496508,0,0,...,0,0,0,0,0,0,0,0,0,5


In [15]:
log_feature = log_featurex

# Data Wrangling for Feature Selection

In [16]:
# LOCATION AND NUMBER OF FAULT SEVERITY'S HASH LOOKUP TABLE
joined = pd.merge(train, severity_type, left_index=True, right_index=True, how='left')
joined.drop(['id_y'],inplace=True,axis=1)
#joined['loc_sev_type'] = joined.apply(lambda row: (str(row['location']) + (str(row['severity_type']))),axis=1)
#joined = pd.get_dummies(joined,columns = ['severity_type'])

joined['frequency0'] = joined.fault_severity
joined['frequency1'] = joined.fault_severity
joined['frequency2'] = joined.fault_severity
def increment_freq(x,col):
    if col == x:
        return 1
    else:
        return 0

joined['frequency0'] = joined['frequency0'].apply(increment_freq,args=(0,))
joined['frequency1'] = joined['frequency1'].apply(increment_freq,args=(1,))
joined['frequency2'] = joined['frequency2'].apply(increment_freq,args=(2,))

joined = joined.sort_index()
joinedy = joined.groupby(by='location')['frequency0'].sum()
joinedy1 = joined.groupby(by='location')['frequency1'].sum()
joinedy2 = joined.groupby(by='location')['frequency2'].sum()

LOC_DICT = {0:joinedy.to_dict(),1:joinedy1.to_dict(),2:joinedy2.to_dict()}

for x,value in joined.iterrows():
        frq = LOC_DICT[0][str(value['location'])]
        joined.set_value(x,'frequency0',frq)
        frq = LOC_DICT[1][str(value['location'])]
        joined.set_value(x,'frequency1',frq)
        frq = LOC_DICT[2][str(value['location'])]
        joined.set_value(x,'frequency2',frq)

joined.columns

Index([u'id_x', u'location', u'fault_severity', u'severity_type',
       u'frequency0', u'frequency1', u'frequency2'],
      dtype='object')

In [17]:
print joined

        id_x location  fault_severity severity_type  frequency0  frequency1  \
id                                                                            
1          1      601               1             1           4           5   
5          5      460               0             2          12          11   
6          6      332               1             2           2           2   
8          8      243               0             2           7           0   
13        13      418               0             2           4           4   
19        19      644               1             1          11           8   
20        20       79               0             2           2           6   
23        23      257               0             2           3           3   
24        24      367               0             4           2           3   
26        26      238               0             4           2           1   
27        27      793               0             1 

In [30]:
joined1 = pd.merge(joined, event_type, left_index=True, right_index=True, how='left')
joined1.shape

(7381, 61)

In [31]:
joined2 = pd.merge(joined1, resource_type, left_index=True, right_index=True, how='left')
joined2.shape


(7381, 72)

In [32]:
joined3 = pd.merge(joined2, log_feature, left_index=True, right_index=True, how='left')
joined3.shape
del joined3['id_x']

In [33]:
#joined3['location'] = joined3['location'].apply(lambda x:LOC[x])


In [34]:
joined3 = pd.get_dummies(joined3,columns=['location'])

In [35]:
#LOG NORMALIZATION
joined3['frequency0'] = joined3['frequency0'] .apply(lambda x: np.log(x+1))
joined3['frequency1'] = joined3['frequency1'] .apply(lambda x: np.log(x+1))
joined3['frequency2'] = joined3['frequency2'] .apply(lambda x: np.log(x+1))
joined3['resource_type']= joined3['resource_type'] .apply(lambda x: np.log(x+1))
joined3['event_type']= joined3['event_type'] .apply(lambda x: np.log(x+1))


In [36]:
joined3[:20]

,fault_severity,severity_type,frequency0,frequency1,frequency2,event_type,event_type_1,event_type_10,event_type_11,event_type_12,...,location_987,location_989,location_99,location_990,location_991,location_994,location_995,location_996,location_998,location_999
id,,,,,,,,,,,,,,,,,,,,,
1,1,1,1.609438,1.791759,0.693147,1.098612,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
5,0,2,2.564949,2.484907,0.000000,1.098612,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,1,2,1.098612,1.098612,0.000000,0.693147,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,2,2.079442,0.000000,0.000000,1.098612,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
13,0,2,1.609438,1.609438,0.000000,1.098612,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
19,1,1,2.484907,2.197225,1.386294,1.098612,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
20,0,2,1.098612,1.945910,0.000000,1.386294,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0
23,0,2,1.386294,1.386294,0.000000,1.098612,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
24,0,4,1.098612,1.386294,0.000000,0.693147,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [37]:
# TEST DATA TRANSFORMATION
# LOCATION AND NUMBER OF FAULT SEVERITY'S HASH LOOKUP TABLE
joined_tst = pd.merge(test, severity_type, left_index=True, right_index=True, how='left')
joined_tst.drop(['id_y'],inplace=True,axis=1)
#joined['loc_sev_type'] = joined.apply(lambda row: (str(row['location']) + (str(row['severity_type']))),axis=1)
#joined_tst = pd.get_dummies(joined_tst,columns = ['severity_type'])

joined_tst['frequency0'] = 0
joined_tst['frequency1'] = 0
joined_tst['frequency2'] = 0

for x,value in joined_tst.iterrows():
        if value['location']in LOC_DICT[0]:
            frq = LOC_DICT[0][str(value['location'])]
            joined_tst.set_value(x,'frequency0',frq)
        if value['location']in LOC_DICT[1]:
            frq = LOC_DICT[1][str(value['location'])]
            joined_tst.set_value(x,'frequency1',frq)
        if value['location']in LOC_DICT[2]:
            frq = LOC_DICT[2][str(value['location'])]
            joined_tst.set_value(x,'frequency2',frq)

joined_tst.columns

Index([u'id_x', u'location', u'severity_type', u'frequency0', u'frequency1',
       u'frequency2'],
      dtype='object')

In [38]:
joined1_tst = pd.merge(joined_tst, event_type, left_index=True, right_index=True, how='left')
joined1_tst.columns

Index([u'id_x', u'location', u'severity_type', u'frequency0', u'frequency1',
       u'frequency2', u'event_type', u'event_type_1', u'event_type_10',
       u'event_type_11', u'event_type_12', u'event_type_13', u'event_type_14',
       u'event_type_15', u'event_type_17', u'event_type_18', u'event_type_19',
       u'event_type_2', u'event_type_20', u'event_type_21', u'event_type_22',
       u'event_type_23', u'event_type_24', u'event_type_25', u'event_type_26',
       u'event_type_27', u'event_type_28', u'event_type_29', u'event_type_3',
       u'event_type_30', u'event_type_31', u'event_type_32', u'event_type_33',
       u'event_type_34', u'event_type_35', u'event_type_36', u'event_type_37',
       u'event_type_38', u'event_type_39', u'event_type_4', u'event_type_40',
       u'event_type_41', u'event_type_42', u'event_type_43', u'event_type_44',
       u'event_type_45', u'event_type_46', u'event_type_47', u'event_type_48',
       u'event_type_49', u'event_type_5', u'event_type_50', u'ev

In [39]:
joined2_tst = pd.merge(joined1_tst, resource_type, left_index=True, right_index=True, how='left')
joined2_tst[:4]

,id_x,location,severity_type,frequency0,frequency1,frequency2,event_type,event_type_1,event_type_10,event_type_11,...,resource_type_1,resource_type_10,resource_type_2,resource_type_3,resource_type_4,resource_type_5,resource_type_6,resource_type_7,resource_type_8,resource_type_9
id,,,,,,,,,,,,,,,,,,,,,
11066,11066,481,2,16,3,1,2,0,0,0,...,0,0,1,0,0,0,0,0,0,0
18000,18000,962,1,13,12,20,2,0,0,1,...,0,0,0,0,0,0,0,0,1,0
16964,16964,491,2,15,0,0,2,0,0,0,...,0,0,1,0,0,0,0,0,0,0
4795,4795,532,5,2,1,0,2,0,1,0,...,0,0,0,1,0,0,0,0,0,1


In [40]:
joined3_tst = pd.merge(joined2_tst, log_feature, left_index=True, right_index=True, how='left')
joined3_tst.shape
del joined3_tst['id_x']
def apply_abcd(x):
    if x in LOC.keys():
        return LOC[x]
    else:
        return 0
#joined3_tst['location'] = joined3_tst['location'].apply(apply_abcd,)


In [41]:
joined3_tst = pd.get_dummies(joined3_tst,columns=['location'])
# joined3_tst[:4]


In [42]:
#LOG NORMALIZATION
joined3_tst['frequency0'] = joined3_tst['frequency0'] .apply(lambda x: np.log(x+1))
joined3_tst['frequency1'] = joined3_tst['frequency1'] .apply(lambda x: np.log(x+1))
joined3_tst['frequency2'] = joined3_tst['frequency2'] .apply(lambda x: np.log(x+1))
joined3_tst['resource_type']= joined3_tst['resource_type'] .apply(lambda x: np.log(x+1))
joined3_tst['event_type']= joined3_tst['event_type'] .apply(lambda x: np.log(x+1))


In [43]:
print joined3_tst.shape
print joined3.shape

for j,val2 in joined3_tst.iteritems():
    if j not in joined3.columns.values:
#         del joined3_tst[j]
        joined3_tst.drop([j],inplace=True,axis=1,errors='ignore')

# for i,value in joined3.iteritems():
print joined3_tst.shape
print joined3.shape

(11171, 1495)
(7381, 1386)
(11171, 1298)
(7381, 1386)


In [44]:
for j,val2 in joined3.iteritems():
    if j == 'fault_severity':
        print 'x'
        #do nothing
    elif j not in joined3_tst.columns.values:
        print j
        del joined3[j]

# for i,value in joined3.iteritems():
print joined3_tst.shape
print joined3.shape

x
location_1016
location_1027
location_1070
location_1087
location_1110
location_1126
location_113
location_180
location_188
location_196
location_214
location_225
location_241
location_247
location_250
location_254
location_259
location_267
location_277
location_285
location_287
location_291
location_293
location_304
location_319
location_329
location_331
location_333
location_339
location_346
location_355
location_357
location_362
location_37
location_389
location_391
location_399
location_407
location_423
location_424
location_487
location_5
location_511
location_512
location_516
location_523
location_524
location_529
location_531
location_537
location_548
location_551
location_555
location_556
location_559
location_56
location_560
location_562
location_57
location_573
location_578
location_588
location_597
location_614
location_617
location_621
location_624
location_633
location_635
location_65
location_67
location_694
location_715
location_716
location_717
location_718
location_74

In [46]:
# !pip install xgboost
#!pip install xgboost --upgrade

In [50]:
joined3

,fault_severity,severity_type,frequency0,frequency1,frequency2,event_type,event_type_1,event_type_10,event_type_11,event_type_12,...,location_987,location_989,location_99,location_990,location_991,location_994,location_995,location_996,location_998,location_999
id,,,,,,,,,,,,,,,,,,,,,
1,1,1,1.609438,1.791759,0.693147,1.098612,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
5,0,2,2.564949,2.484907,0.000000,1.098612,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,1,2,1.098612,1.098612,0.000000,0.693147,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,2,2.079442,0.000000,0.000000,1.098612,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
13,0,2,1.609438,1.609438,0.000000,1.098612,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
19,1,1,2.484907,2.197225,1.386294,1.098612,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
20,0,2,1.098612,1.945910,0.000000,1.386294,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0
23,0,2,1.386294,1.386294,0.000000,1.098612,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
24,0,4,1.098612,1.386294,0.000000,0.693147,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [51]:
# This function takes in a dataframe, and returns a list of dataframes partitioned 
def Split_by_sev_type(df):
    if 'severity_type' not in df.columns:
        return 'NO SEV TYPE COLUMN'
    else:
        list_df = []
        dframe = df.copy(deep=True)
#         for x in range(1,6,1):
        list_df.append(dframe[dframe['severity_type'] == str(1)].append(dframe[dframe['severity_type'] == str(3)]).append(dframe[dframe['severity_type'] == str(5)]))
        list_df.append(dframe[dframe['severity_type'] == str(2)])
        list_df.append(dframe[dframe['severity_type'] == str(4)])
        print len(list_df[0]) + len(list_df[1]) + len(list_df[2])
        return list_df

In [52]:
trainX_sev_types = Split_by_sev_type(joined3)
testX_sev_types = Split_by_sev_type(joined3_tst)

7381
11171


In [53]:
# X = pd.DataFrame(joined3)
# X.sort_index(axis=0)
# Y = X['fault_severity']
# del X['fault_severity']
# X[:20]

X = []
Y = []
for x,y in enumerate(trainX_sev_types):
    X.append(y)
    Y.append(y['fault_severity'])
    del y['fault_severity']
# Y = X['fault_severity']
# del X['fault_severity']

In [54]:
X

[      severity_type  frequency0  frequency1  frequency2  event_type  \
 id                                                                    
 1                 1    1.609438    1.791759    0.693147    1.098612   
 19                1    2.484907    2.197225    1.386294    1.098612   
 27                1    4.025352    0.000000    0.000000    0.693147   
 53                1    4.094345    0.693147    0.000000    1.386294   
 57                1    1.791759    2.484907    2.564949    1.098612   
 59                1    1.791759    0.000000    0.000000    0.693147   
 60                1    3.178054    1.945910    1.609438    1.098612   
 67                1    3.433987    3.332205    3.367296    1.098612   
 69                1    1.945910    1.609438    0.000000    0.693147   
 79                1    1.945910    0.693147    0.000000    1.098612   
 84                1    1.386294    1.945910    1.791759    1.098612   
 90                1    1.791759    2.197225    0.693147    0.69

In [55]:
trainX = []
testX = []
testY = []
trainY = []
for x in range(0,3,1):
    print x
    print len(X[x])
    print len(Y[x])
    a, b, c, d = cross_validation.train_test_split(X[x], Y[x], train_size=0.60, test_size=0.4, random_state=9)
    trainX.append(a)
    testX.append(b)
    testY.append(d)
    trainY.append(c)
print trainX

0
3402
3402
1
3591
3591
2
388
388
[      severity_type  frequency0  frequency1  frequency2  event_type  \
id                                                                    
13421             1    0.693147    1.098612    0.000000    0.693147   
17029             1    1.386294    1.098612    0.000000    0.693147   
11781             1    2.079442    0.000000    0.000000    0.693147   
2352              1    3.663562    0.000000    0.000000    0.693147   
12291             1    2.197225    0.000000    0.000000    0.693147   
3081              1    0.000000    1.609438    0.693147    1.098612   
17185             1    3.044522    2.708050    3.135494    1.098612   
7610              1    3.178054    2.833213    2.708050    1.098612   
10990             1    3.135494    2.708050    2.944439    1.098612   
11430             1    3.332205    1.791759    0.000000    1.386294   
14004             1    3.044522    2.708050    3.135494    1.098612   
17273             1    0.000000    0.69314

In [ ]:
# load the client interface

from IPython import parallel

clients = parallel.Client()
clients.block = True   # wait for calculations to finish
print clients.ids      # client process ids
dview = clients.direct_view()

In [ ]:
%%px 
import sys 
sys.path.append('/home/shivam6294/anaconda2/lib/python2.7/site-packages/') 
from numpy import *
# load libraries on clusters
from sklearn import *
from pickle import *
from sklearn.metrics import *
from sklearn.grid_search import *
from sklearn.ensemble import *
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import *
from operator import itemgetter
import xgboost

In [ ]:
# get the load-balanced scheduler
lview = clients.load_balanced_view()


# XGBOOST
 * The idea is to train 2 XGBoost Classifiers , on first two data splits, and Random Forests on the last split.
 * Data has been split, on 3 different severity types

# Data Split 0

## Training

In [73]:
from operator import itemgetter

def report(grid_scores, n_top=3):
    top_scores = sorted(grid_scores, key=itemgetter(1), reverse=True)[:n_top]
    for i, score in enumerate(top_scores):
        print("Model with rank: {0}".format(i + 1))
        print("Mean validation score: {0:.3f} (std: {1:.3f})".format(
              score.mean_validation_score,
              np.std(score.cv_validation_scores)))
        print("Parameters: {0}".format(score.parameters))
        print("")

In [74]:
# XGBoost
# Train uncalibrated random forest classifier on whole train and validation
# data and evaluate on test data
from sklearn.metrics import *
from sklearn.grid_search import *
def XGBoost_ClassifierCV(data,grid):
    xgb_model = xgb.XGBClassifier()
    print xgb_model.get_params().keys()
    xgb_model.set_params(**grid)
    print xgb_model.get_params().values()
#     xgbmodel1 = xgb_model.fit(data['trainX'],data['trainY'])
#     predictions = xgbmodel1.predict_proba(data['testX'])
#     score = log_loss(data['testY'], predictions)
    grid_search_ = GridSearchCV(xgb_model, param_grid=grid,cv=3,scoring = 'log_loss',n_jobs=-1)
    grid_search_.fit(data['trainX'], data['trainY'])
    print("GridSearchCV took %d candidate parameter settings."
      % len(grid_search_.grid_scores_))
    report(grid_search_.grid_scores_)
#     print(score)

# run grid search

In [ ]:
data = {'trainX': trainX[0].append(testX[0]).as_matrix(), 'trainY': trainY[0].append(testY[0]).as_matrix()}
grid ={'objective':['multi:softprob'],'n_estimators':[160,180,200,220],'gamma':[0.0,0.1],'max_depth':[4,5,6]}
XGBoost_ClassifierCV(data,grid)

['reg_alpha', 'colsample_bytree', 'silent', 'colsample_bylevel', 'scale_pos_weight', 'learning_rate', 'missing', 'max_delta_step', 'nthread', 'base_score', 'n_estimators', 'subsample', 'reg_lambda', 'seed', 'min_child_weight', 'objective', 'max_depth', 'gamma']
[0, 1, True, 1, 1, 0.1, None, 0, -1, 0.5, [160, 180, 200, 220], 1, 1, 0, 1, ['multi:softprob'], [4, 5, 6], [0.0, 0.1]]


Process PoolWorker-5:
Process PoolWorker-7:
Process PoolWorker-6:
Process PoolWorker-8:
Process PoolWorker-4:
Process PoolWorker-11:
Process PoolWorker-10:
Process PoolWorker-3:
Process PoolWorker-9:
Process PoolWorker-1:
Process PoolWorker-2:
Process PoolWorker-12:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/shivam6294/anaconda2/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
  File "/home/shivam6294/anaconda2/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
  File "/home/shivam6294/anaconda2/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
  File 

KeyboardInterrupt: 

ERROR:tornado.general:Uncaught exception, closing connection.
Traceback (most recent call last):
  File "/home/shivam6294/anaconda2/lib/python2.7/site-packages/zmq/eventloop/zmqstream.py", line 414, in _run_callback
    callback(*args, **kwargs)
  File "/home/shivam6294/anaconda2/lib/python2.7/site-packages/tornado/stack_context.py", line 275, in null_wrapper
    return fn(*args, **kwargs)
  File "/home/shivam6294/anaconda2/lib/python2.7/site-packages/ipykernel/kernelbase.py", line 260, in dispatcher
    return self.dispatch_shell(stream, msg)
  File "/home/shivam6294/anaconda2/lib/python2.7/site-packages/ipykernel/kernelbase.py", line 212, in dispatch_shell
    handler(stream, idents, msg)
  File "/home/shivam6294/anaconda2/lib/python2.7/site-packages/ipykernel/kernelbase.py", line 396, in execute_request
    self._abort_queues()
  File "/home/shivam6294/anaconda2/lib/python2.7/site-packages/ipykernel/kernelbase.py", line 614, in _abort_queues
    self._abort_queue(stream)
  File "/ho

In [75]:
# XGBoost
from sklearn.metrics import *
from sklearn.grid_search import *
def XGBoost_Classifier_PREDICT(data,grid):
    xgb_model = xgb.XGBClassifier()
    print xgb_model.get_params().keys()
    xgb_model.set_params(**grid)
    print xgb_model.get_params().values()
    xgbmodel1 = xgb_model.fit(data['trainX'],data['trainY'])
    print data['trainX'].shape
    print data['trainY'].shape
    predictions = xgbmodel1.predict_proba(data['testX'])
    print data['testX'].shape
    print predictions.shape
    return predictions

In [77]:
# # XGBoost
# from sklearn.metrics import *
# from sklearn.grid_search import *
# from sklearn.cross_validation import cross_val_predict
# def XGBoost_Cross_val_predict(data,grid):
#     xgb_model1 = xgb.XGBClassifier()
#     print xgb_model1.get_params().keys()
#     xgb_model1.set_params(**grid)
#     xgb_model = proba_xgb(xgb_model1)
#     print xgb_model.get_params().values()
# #     xgbmodel1 = xgb_model.fit(data['trainX'],data['trainY'])
    
# #     print data['trainX'].shape
# #     print data['trainY'].shape
# #     predictions = xgbmodel1.predict_proba(data['testX'])
#     predictions = cross_val_predict(xgb_model, data['trainX'], y=data['trainY'], cv=2, n_jobs=-1, verbose=0, fit_params=None)
#     print predictions.shape
#     return predictions



In [78]:
# class proba_xgb(xgb.XGBClassifier):
#     def predict(X):
#         return xgb.XGBClassifier.predict_proba(self, X)


In [79]:
if 'severity_type' in testX_sev_types[0].columns.values:
    testX_sev_types[0] = pd.get_dummies(testX_sev_types[0],columns=['severity_type'])
if 'severity_type' in testX[0].columns.values:
    testX[0] = pd.get_dummies(testX[0],columns=['severity_type'])
if 'severity_type' in trainX[0].columns.values:
    trainX[0] = pd.get_dummies(trainX[0],columns=['severity_type'])


In [80]:
type(trainY[0].append(testY[0]))

pandas.core.series.Series

In [115]:
data = {'trainX': trainX[0].append(testX[0]), 'trainY': trainY[0].append(testY[0]), 'testX':trainX[0].append(testX[0])}
grid ={'objective':'multi:softprob','n_estimators':160,'gamma':0.0,'max_depth':4}
data0 = trainX[0].append(testX[0]).index.values
preds0 = XGBoost_Classifier_PREDICT(data,grid)

['reg_alpha', 'colsample_bytree', 'silent', 'colsample_bylevel', 'scale_pos_weight', 'learning_rate', 'missing', 'max_delta_step', 'nthread', 'base_score', 'n_estimators', 'subsample', 'reg_lambda', 'seed', 'min_child_weight', 'objective', 'max_depth', 'gamma']
[0, 1, True, 1, 1, 0.1, None, 0, -1, 0.5, 160, 1, 1, 0, 1, 'multi:softprob', 4, 0.0]
(3402, 1300)
(3402,)
(3402, 1300)
(3402, 3)


# Data Split 1


## Cross Validation

In [66]:
data = {'trainX': trainX[1].append(testX[1]).as_matrix(), 'trainY': trainY[1].append(testY[1]).as_matrix()}
grid ={'objective':['multi:softprob'],'n_estimators':[140,220],'gamma':[0.0],'max_depth':[4]}
data1, preds1 = XGBoost_Classifier(data,grid)

NameError: name 'XGBoost_Classifier' is not defined

## Training

In [116]:
if 'severity_type' in testX_sev_types[1].columns.values:
    testX_sev_types[1] = testX_sev_types[1].drop(['severity_type'],axis=1)
if 'severity_type' in testX[1].columns.values:
    testX[1] = testX[1].drop(['severity_type'],axis=1)
if 'severity_type' in trainX[1].columns.values:
    trainX[1] = trainX[1].drop(['severity_type'],axis=1)

In [117]:
testX_sev_types[1]

,frequency0,frequency1,frequency2,event_type,event_type_1,event_type_10,event_type_11,event_type_12,event_type_13,event_type_14,...,location_987,location_989,location_99,location_990,location_991,location_994,location_995,location_996,location_998,location_999
id,,,,,,,,,,,,,,,,,,,,,
11066,2.833213,1.386294,0.693147,1.098612,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
16964,2.772589,0.000000,0.000000,1.098612,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3392,3.332205,2.197225,3.401197,0.693147,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2881,2.302585,1.098612,0.000000,1.098612,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
15795,1.098612,1.098612,0.000000,1.098612,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4315,0.693147,0.000000,0.000000,0.693147,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6764,3.663562,0.693147,0.000000,1.098612,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
13039,3.332205,1.791759,0.000000,1.098612,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
14638,0.693147,0.000000,0.000000,1.098612,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [119]:
data = {'trainX': trainX[1].append(testX[1]), 'trainY': trainY[1].append(testY[1]), 'testX':trainX[1].append(testX[1])}
grid ={'objective':'multi:softprob','n_estimators':200,'gamma':0.0,'max_depth':4}
data1 = trainX[1].append(testX[1]).index.values
preds1 = XGBoost_Classifier_PREDICT(data,grid)

['reg_alpha', 'colsample_bytree', 'silent', 'colsample_bylevel', 'scale_pos_weight', 'learning_rate', 'missing', 'max_delta_step', 'nthread', 'base_score', 'n_estimators', 'subsample', 'reg_lambda', 'seed', 'min_child_weight', 'objective', 'max_depth', 'gamma']
[0, 1, True, 1, 1, 0.1, None, 0, -1, 0.5, 200, 1, 1, 0, 1, 'multi:softprob', 4, 0.0]
(3591, 1297)
(3591,)
(3591, 1297)
(3591, 3)


In [66]:
print preds1

[[  9.48284030e-01   5.12420833e-02   4.73834167e-04]
 [  9.99615192e-01   3.75530857e-04   9.22922572e-06]
 [  3.54221553e-01   6.33230686e-01   1.25477491e-02]
 ..., 
 [  2.89035469e-01   7.10658491e-01   3.05989728e-04]
 [  7.07051396e-01   2.92724252e-01   2.24327683e-04]
 [  9.98646796e-01   1.33887096e-03   1.43092575e-05]]


In [67]:
data1

array([11066, 16964,  3392, ..., 13628,  9406,  7520])

In [68]:
trainX[1].append(testX[1]).shape

(3591, 1297)

# Data Split 2

In [80]:
#CROSS VALIDATION PARAMS

#Best for index 1 = 0.377
from sklearn.grid_search import GridSearchCV, RandomizedSearchCV
params = {'n_estimators': [200,220,250]}
clf = ensemble.RandomForestClassifier(max_depth=20,max_features=400,n_jobs=4)
# clf.fit(trainX,trainY)
# clf_probs = clf.predict_proba(testX)
# score = log_loss(testY, clf_probs)

# run grid search
grid_search = GridSearchCV(clf, param_grid=params,cv=3,scoring = 'log_loss',n_jobs=-1)
grid_search.fit(trainX[2].append(testX[2]), trainY[2].append(testY[2]))

print("GridSearchCV took %d candidate parameter settings."
      % len(grid_search.grid_scores_))
report(grid_search.grid_scores_)

GridSearchCV took 3 candidate parameter settings.


NameError: name 'report' is not defined

In [69]:
from operator import itemgetter

def report(grid_scores, n_top=3):
    top_scores = sorted(grid_scores, key=itemgetter(1), reverse=True)[:n_top]
    for i, score in enumerate(top_scores):
        print("Model with rank: {0}".format(i + 1))
        print("Mean validation score: {0:.3f} (std: {1:.3f})".format(
              score.mean_validation_score,
              np.std(score.cv_validation_scores)))
        print("Parameters: {0}".format(score.parameters))
        print("")


## Training Data


In [129]:
#Best for index 1 = 0.377
from sklearn.grid_search import GridSearchCV, RandomizedSearchCV
params = {'n_estimators': [220]}
clf = ensemble.RandomForestClassifier(max_depth=20,max_features=400,n_jobs=4)
clf.fit(trainX[2].append(testX[2]), trainY[2].append(testY[2]))
clf_probs = clf.predict_proba(trainX[2].append(testX[2]))
data2= trainX[2].append(testX[2]).index.values
preds2 = clf_probs

In [130]:
len(preds2)

388

In [131]:
preds2_ = []
for x,y in enumerate(preds2):
    preds2_.append(np.append(y, 0.0))
preds2_

[array([ 1.,  0.,  0.]),
 array([ 0.8,  0.2,  0. ]),
 array([ 1.,  0.,  0.]),
 array([ 1.,  0.,  0.]),
 array([ 1.,  0.,  0.]),
 array([ 1.,  0.,  0.]),
 array([ 0.5,  0.5,  0. ]),
 array([ 0.8,  0.2,  0. ]),
 array([ 1.,  0.,  0.]),
 array([ 1.,  0.,  0.]),
 array([ 0.1,  0.9,  0. ]),
 array([ 1.,  0.,  0.]),
 array([ 1.,  0.,  0.]),
 array([ 1.,  0.,  0.]),
 array([ 1.,  0.,  0.]),
 array([ 1.,  0.,  0.]),
 array([ 0.6,  0.4,  0. ]),
 array([ 1.,  0.,  0.]),
 array([ 1.,  0.,  0.]),
 array([ 1.,  0.,  0.]),
 array([ 1.,  0.,  0.]),
 array([ 0.4,  0.6,  0. ]),
 array([ 1.,  0.,  0.]),
 array([ 1.,  0.,  0.]),
 array([ 1.,  0.,  0.]),
 array([ 1.,  0.,  0.]),
 array([ 1.,  0.,  0.]),
 array([ 1.,  0.,  0.]),
 array([ 1.,  0.,  0.]),
 array([ 0.2,  0.8,  0. ]),
 array([ 1.,  0.,  0.]),
 array([ 1.,  0.,  0.]),
 array([ 1.,  0.,  0.]),
 array([ 1.,  0.,  0.]),
 array([ 1.,  0.,  0.]),
 array([ 0.2,  0.8,  0. ]),
 array([ 1.,  0.,  0.]),
 array([ 1.,  0.,  0.]),
 array([ 1.,  0.,  0.]),
 

In [123]:
print data0.shape
print data1.shape

(3402,)
(3591,)


# WRITING OUTPUT for 3 splits
### PREDICTION PROBABILITIES ON THE TEST SET

In [170]:
# write a kaggle submission file for "whats cooking"
def write_csv_kaggle_(fname, ids, target):
    # header
    tmp = [['id', 'predict_0', 'predict_1', 'predict_2']]
    
    for i in range(0,len(ids),1):
        # add a row (id and class prediction)
        tmp.append([ids[i], target[i][0], target[i][1],target[i][2]])
        
    # write CSV file
    f = open(fname, 'wb')
    writer = csv.writer(f)
    writer.writerows(tmp)
    f.close()

In [134]:
labels = []
labels = list(data0) + list(data1)
labels = labels + list(data2)
predictions = list(preds0) + list(preds1)
predictions = predictions + list(preds2_)



In [135]:
print len(labels)
print len(predictions)

DICT_classifier1 = {}

for i,j in enumerate(labels):
    DICT_classifier1[j] = predictions[i]

7381
7381


In [136]:
df_classifier1 = pd.DataFrame.from_dict(DICT_classifier1,orient='index')
df_classifier1.columns = ['clf1_0','clf1_1','clf1_2']
df_classifier1

,clf1_0,clf1_1,clf1_2
1,0.236312,0.703024,0.060664
5,0.627439,0.372339,0.000222
6,0.147017,0.852750,0.000233
8,0.999335,0.000654,0.000011
13,0.309640,0.690059,0.000302
19,0.259530,0.682055,0.058415
20,0.761594,0.238226,0.000180
23,0.425996,0.573726,0.000278
24,1.000000,0.000000,0.000000
26,1.000000,0.000000,0.000000


In [137]:
# df_classifier1.to_pickle('df_classifier1.p')

In [138]:
# write_csv_kaggle_('3_models.csv',labels,predictions)

# Location Clustering
* We're gonna try and cluster similar locations together
* Instinct: Try and capitalise on network disruption patterns that arise due to location

In [18]:
# loc_cluster1 = pd.get_dummies(train,columns = ['fault_severity'])
# loc_cluster1 =loc_cluster1[['location','fault_severity_0','fault_severity_1','fault_severity_2']].groupby(by='location').sum()
# # loc_cluster1 = loc_cluster1.apply(lambda x : np.log(x+1))
# loc_cluster1 

In [19]:
# from sklearn.metrics import *
# dist = 1 - pairwise_distances(loc_cluster1)
# print dist

# from scipy.cluster.hierarchy import ward, dendrogram

# linkage_matrix = ward(dist) #define the linkage_matrix using ward clustering pre-computed distances
# linkage_matrix

# fig, ax = plt.subplots(figsize=(100, 200)) # set size
# ax = dendrogram(linkage_matrix, orientation="right", labels=loc_cluster1.index.values);

# plt.tick_params(\
#     axis= 'x',          # changes apply to the x-axis
#     which='both',      # both major and minor ticks are affected
#     bottom='off',      # ticks along the bottom edge are off
#     top='off',         # ticks along the top edge are off
#     labelbottom='off')

# plt.tight_layout() #show plot with tight layout

# #uncomment below to save figure
# plt.savefig('ward_clusters_telstra_location.png') #save figure as ward_clusters

# ax['leaves']
# dist
# from sklearn.cluster import *
# model = AgglomerativeClustering(n_clusters=5,linkage="complete",affinity='precomputed')
# loc_prediction = model.fit_predict(dist)
# loc_prediction
# print loc_prediction.shape
# location_arr = loc_cluster1.index.values
# LOC = {}
# for x,value in enumerate(loc_prediction):
#     LOC[location_arr[x]] = value

#  df1 = pd.DataFrame.from_dict(LOC,orient='index')
# loc_cluster1 = pd.merge(loc_cluster1, df1, left_index=True, right_index=True, how='left')
# loc_cluster1.columns = [u'fault_severity_0', u'fault_severity_1', u'fault_severity_2', u'0']

# loc_cluster1

# from mpl_toolkits.mplot3d import Axes3D
# threedee = plt.figure().gca(projection='3d')
# threedee.set_xlabel('sev 0')
# threedee.set_ylabel('sev 1')
# threedee.set_zlabel('sev 2')
# mkr_dict = {'0.0': 'b', '1.0': 'g', '2.0': 'r','3.0':'k','4.0':'m'}
# for x,value in loc_cluster1.iterrows():
#     threedee.scatter(value['fault_severity_0'], value['fault_severity_1'], value['fault_severity_2'],color = mkr_dict[str(value['0'])])

# threedee.view_init(30, 120)
# plt.show()